In [43]:
import json
import csv
import os
from github import Github
import datetime
import pandas as pd

# GitHub Login mittels Token
g = Github(os.environ['GITHUBTOKEN'])

# JSON Daten laden, Variablen setzen
with open('github_repos.json', encoding='utf-8') as file:
    githubrepos = json.load(file)

In [71]:
repo = g.get_repo(githubrepos['repos'][0])

heute = datetime.datetime.now()
last_year_date = heute - datetime.timedelta(days=365)

def get_stats_from_repo(repo):

    commit_activities = repo.get_stats_commit_activity()
    last_years_commits = 0
    # Alle Commits der letzten 12 Monate zusammenzählen
    if commit_activities != None:
        for week in commit_activities:
            last_years_commits += week.total
        
    contributors = repo.get_stats_contributors()
    if contributors != None:
        num_contributors = len(contributors)

    # returns growth rate of number of commits in the last 3 months compared to the 9 months before
    def get_recent_relative_commit_activity(repo):
        stats_participation = repo.get_stats_participation()
        return (sum(stats_participation.all[-12:])*40) / (12*sum(stats_participation.all[0:-12]))
    
    def get_recent_relative_issue_activity(repo, state):
        last_quarter = repo.get_issues(state=state, since=heute-datetime.timedelta(days=90)).totalCount
        last_year = repo.get_issues(state=state, since=heute-datetime.timedelta(days=360)).totalCount
        return (last_quarter*9)/((last_year-last_quarter)*3)

        

    stats = {
        'stars': repo.stargazers_count,
        'num_watchers': repo.subscribers_count,
        'num_forks': repo.forks_count,
        'num_contributors': num_contributors,
        'num_pull_requests': repo.get_pulls().totalCount,
        'last_years_commits': last_years_commits,
        'num_commits': repo.size,                # Diese Variable stimmt nicht: es wird teilweise die Anzahl Commits, teilweise auch was ganz anderes zurückgegeben
        'all_issues': repo.get_issues(state="all").totalCount,
        'commit_growth_rate': get_recent_relative_commit_activity(repo),
        'open_issue_growth_rate': get_recent_relative_issue_activity(repo, 'open'),
        'closed_issue_growth_rate': get_recent_relative_issue_activity(repo, 'closed')
        
    }
    return stats
stats = get_stats_from_repo(repo)
stats

{'stars': 67939,
 'num_watchers': 3237,
 'num_forks': 17959,
 'num_contributors': 100,
 'num_pull_requests': 476,
 'last_years_commits': 3021,
 'num_commits': 267193,
 'all_issues': 39227,
 'commit_growth_rate': 0.8976464148877942,
 'open_issue_growth_rate': 8.52093023255814,
 'closed_issue_growth_rate': 1.1908893709327548}

In [72]:
stats = {}
for repo in githubrepos['repos']:
    stats[repo] = get_stats_from_repo(g.get_repo(repo)) 
stats

{'angular/angular': {'stars': 67939,
  'num_watchers': 3237,
  'num_forks': 17959,
  'num_contributors': 100,
  'num_pull_requests': 476,
  'last_years_commits': 3021,
  'num_commits': 267193,
  'all_issues': 39227,
  'commit_growth_rate': 0.8976464148877942,
  'open_issue_growth_rate': 8.52093023255814,
  'closed_issue_growth_rate': 1.1908893709327548},
 'facebook/react': {'stars': 159193,
  'num_watchers': 6782,
  'num_forks': 31603,
  'num_contributors': 100,
  'num_pull_requests': 134,
  'last_years_commits': 1079,
  'num_commits': 160669,
  'all_issues': 20046,
  'commit_growth_rate': 0.6512989388949872,
  'open_issue_growth_rate': 2.9363957597173145,
  'closed_issue_growth_rate': 0.7256883931683513},
 'vuejs/vue': {'stars': 175316,
  'num_watchers': 6343,
  'num_forks': 27217,
  'num_contributors': 99,
  'num_pull_requests': 204,
  'last_years_commits': 64,
  'num_commits': 27603,
  'all_issues': 11159,
  'commit_growth_rate': 1.08843537414966,
  'open_issue_growth_rate': 1.93373

In [73]:
df = pd.DataFrame.from_dict(stats, orient='index')
df

,stars,num_watchers,num_forks,num_contributors,num_pull_requests,last_years_commits,num_commits,all_issues,commit_growth_rate,open_issue_growth_rate,closed_issue_growth_rate
angular/angular,67939,3237,17959,100,476,3021,267193,39227,0.897646,8.520930,1.190889
facebook/react,159193,6782,31603,100,134,1079,160669,20046,0.651299,2.936396,0.725688
vuejs/vue,175316,6343,27217,99,204,64,27603,11159,1.088435,1.933735,0.953125
aurelia/framework,11516,462,686,96,9,37,2287,947,1.111111,1.125000,3.000000
emberjs/ember.js,21653,961,4217,100,91,555,67413,15608,1.126472,1.491979,0.927481


In [74]:
df.to_csv('output.csv')